In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
artists=pd.read_csv("../Data/info_artists.csv")

In [ ]:
artists

In [ ]:
genres = []
for item in artists["artist_genres"]:
    genres.append(re.findall(r"'(.*?)'",item))
    
artists["artist_genres"]=genres

In [ ]:
artists['len'] = artists['artist_genres'].apply(lambda x: len(x))

In [ ]:
new_artists = artists[artists['len'] != 0 ] 
new_artists.reset_index(drop=True, inplace=True)
new_artists

In [ ]:
dict_genres={}

for item in new_artists["artist_genres"]:
    for genre in item:
        if genre in dict_genres.keys():
            dict_genres[genre]+=1
        else:
            dict_genres[genre]=1
   

In [ ]:
dict_genres

In [ ]:
def get_artist_most_popular_genre(artist_genres: list, filtered_dictionary: dict):
    
    popular = []
    
    for genre in artist_genres:
        if genre in filtered_dictionary.keys():
            popular.append( (filtered_dictionary[genre], genre) )
        else: 
            popular.append( (0, "None") )
    
    max_popular = max(popular)[1]
    return max_popular


In [ ]:
results=[ get_artist_most_popular_genre(item, dict_genres) for item in new_artists['artist_genres'] ]
new_artists["genre"]=results

In [ ]:
new_artists.drop(["artist_genres","len"], axis=1, inplace=True)

In [ ]:
new_artists.sort_values(by=["genre","artist_popularity"])

In [ ]:
#Dropping artist that genre frequency is less than 10
new_artists.genre.value_counts()

In [ ]:
most_popular_genre=[]
for x,y in new_artists.genre.value_counts().items():
    if y > 10:
        most_popular_genre.append(x)
most_popular_genre

In [ ]:
final_artists=new_artists[new_artists["genre"].isin(most_popular_genre)].reset_index(drop=True)
final_artists

In [ ]:
round(pd.pivot_table(final_artists, index="genre", values=["artist_popularity","artist_n_followers"], aggfunc="mean"),0)

In [ ]:
final_artists.to_csv("../Data/final_artists.csv", index=False)